#**MODEL MAKING**

*Import Libraries*

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

*Download & Unzip Dataset*

In [ ]:
# Dataset : http://cb.lk/covid_19
!wget http://cb.lk/covid_19

In [ ]:
!unzip covid_19


In [ ]:
# Checking the total number of images
import glob

print('Train Images-',len(list(glob.glob('/content/CovidDataset/Train/*/*'))))
print('Test Images-',len(list(glob.glob('/content/CovidDataset/Val/*/*'))))



In [ ]:
# Accessing Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Save the data in google drive path
import shutil
source='/content/CovidDataset'
destination="/content/drive/MyDrive/Data Science/Deployment/COVID19"
shutil.move(source, destination)


In [ ]:
# Sample Image check from Covid folder contents 
import glob 
covid=list(glob.glob('/content/drive/MyDrive/Data Science/Deployment/COVID19/CovidDataset/Train/Covid/*'))
PIL.Image.open(str(covid[0]))




*Training & Validation Dataset Creation*

In [ ]:
img_height,img_width=224,224
batch_size=1
# Training Dataset
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/drive/MyDrive/Data Science/Deployment/COVID19/CovidDataset/Train',
  
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
#Validation Dataset
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/drive/MyDrive/Data Science/Deployment/COVID19/CovidDataset/Val',
  image_size=(img_height, img_width),
  batch_size=batch_size)

*Classes*

In [ ]:
#Classes Identification
class_names = train_ds.class_names
num_classes=len(class_names)
print(class_names,num_classes)


In [ ]:
#VISUALIZATION
# import matplotlib.pyplot as plt
# image_batch, label_batch = next(iter(train_ds))# Load data by iter() & going to the next image by next()
# plt.figure(figsize=(10, 10))
# for i in range(9):
#   ax = plt.subplot(3, 3, i + 1)
#   plt.imshow(image_batch[i].numpy().astype("uint8"))
#   label = label_batch[i]
#   plt.title(class_names[label])
#   plt.axis("off")

*Model Making*

In [ ]:

model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes,activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
epochs=10
history = model.fit(
  train_ds,
  #steps_per_epoch = 100,
  validation_data=val_ds,
  epochs=epochs
)

*Save Model*

In [ ]:
tf.keras.models.save_model(model,'/content/drive/MyDrive/Data Science/Deployment/COVID19/model_covid1.hdf5')